LSOA Centroids Create Empty OD Matrix.



In [1]:
import pandas as pd
import geopandas as gpd


import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# time module, to record the time taken to run the code
import time

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Get all London LSOAs:

downloaded from: https://data.london.gov.uk/dataset/2011-boundary-files



In [7]:
lsoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/lsoa-data.csv', encoding='latin1')


/tmp/ipykernel_30425/2031910545.py:1: DtypeWarning: Columns (174,175,176,178,189,253,254) have mixed types. Specify dtype option on import or set low_memory=False.
  lsoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/lsoa-data.csv', encoding='latin1')


In [8]:
# read in column: Lower Super Output Area as a list
lsoa = lsoa_london['Lower Super Output Area'].tolist()

In [10]:
# drop any na values in list:
lsoa = [x for x in lsoa if str(x) != 'nan']

In [11]:
# count how many lsoas:
len(lsoa)

4835

## Get the shapefile of all England LSOAs, clip out only London ones, and collect their lsoa geo-centroid

data for boundaries downloaded from: https://data.london.gov.uk/dataset/2011-boundary-files


In [49]:
import geopandas as gpd

# Load the LSOA shapefile
england_lsoa_shapefile = r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/england_lsoa_2011.shp'
gdf = gpd.read_file(england_lsoa_shapefile)


In [55]:
gdf.head()

,code,name,label,geometry,centroid,lat,lon
0,E01000037,Barking and Dagenham 003B,E09000002E02000004E01000037,"POLYGON ((0.18508 51.56480, 0.18476 51.56454, ...",POINT (0.17951 51.56252),51.562520,0.179507
1,E01000038,Barking and Dagenham 003C,E09000002E02000004E01000038,"POLYGON ((0.17599 51.56476, 0.17601 51.56475, ...",POINT (0.17308 51.56315),51.563148,0.173084
2,E01000039,Barking and Dagenham 003D,E09000002E02000004E01000039,"POLYGON ((0.18318 51.56090, 0.18336 51.56076, ...",POINT (0.17875 51.55272),51.552723,0.178750
3,E01000040,Barking and Dagenham 019A,E09000002E02000020E01000040,"POLYGON ((0.10095 51.53750, 0.10095 51.53750, ...",POINT (0.09745 51.53663),51.536627,0.097454
4,E01000041,Barking and Dagenham 017A,E09000002E02000018E01000041,"POLYGON ((0.12194 51.53924, 0.12194 51.53922, ...",POINT (0.11613 51.53816),51.538159,0.116128


In [48]:
# show gdf coordinate reference system:
gdf.crs

<Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.01, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [51]:
# convert EPSG:27700 to EPSG:4326:
gdf = gdf.to_crs(epsg=4326)

now find the centroids:

In [54]:
# Calculate centroids
gdf['centroid'] = gdf.geometry.centroid

# Extract the lat and long of the centroids
gdf['lat'] = gdf.centroid.y
gdf['lon'] = gdf.centroid.x

/tmp/ipykernel_30425/3775903783.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid
/tmp/ipykernel_30425/3775903783.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y
/tmp/ipykernel_30425/3775903783.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x


In [56]:
# clip out only the london lsoas
lsoa_london = gdf[gdf['code'].isin(lsoa)]

In [58]:
# now plot the london lsoas centroids on a map with folium:
import folium

# Create a map centered on London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)


# Add centroid points to the map, make them red:
for idx, row in lsoa_london.iterrows():
    folium.Marker(location=[row['lat'], row['lon']], icon=folium.Icon(color='red')).add_to(m)

# save the map as an html file
m.save(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/maps/lsoa_london.html')

In [ ]:
m

check total numbers:

In [45]:
lsoa_london.shape

(4835, 7)

In [46]:
lsoa_london.head()

,code,name,label,geometry,centroid,northings,eastings
0,E01000037,Barking and Dagenham 003B,E09000002E02000004E01000037,"POLYGON ((551549.998 187364.637, 551528.633 18...",POINT (551171.300 187099.695),187099.694903,551171.300398
1,E01000038,Barking and Dagenham 003C,E09000002E02000004E01000038,"POLYGON ((550920.362 187341.138, 550921.876 18...",POINT (550724.067 187156.350),187156.350014,550724.066657
2,E01000039,Barking and Dagenham 003D,E09000002E02000004E01000039,"POLYGON ((551431.061 186927.155, 551444.481 18...",POINT (551151.315 186008.750),186008.749694,551151.315469
3,E01000040,Barking and Dagenham 019A,E09000002E02000020E01000040,"POLYGON ((545806.481 184158.048, 545806.760 18...",POINT (545566.921 184054.130),184054.130064,545566.920722
4,E01000041,Barking and Dagenham 017A,E09000002E02000018E01000041,"POLYGON ((547256.387 184393.166, 547256.625 18...",POINT (546856.993 184261.790),184261.789764,546856.992549


In [ ]:
# save code, name, latitude and longitude to a new dataframe
#lsoa_london_df = lsoa_london[['code', 'name', 'northings', 'eastings']]

Now get all centroids and save to a csv:

In [60]:
# save code, name, latitude and longitude to a new dataframe
lsoa_london_df = lsoa_london[['code', 'name', 'lat', 'lon']]

# save to csv
lsoa_london_df.to_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/lsoa_centroids/gld_lsoa_centroids.csv', index=False)